# Micromobility in Minneapolis, MN: Spatiotemporal Prediction of Origins & Destinations

## Notebook 2: Spatiotemporal Pattern Mining & Analysis

### Luke Zaruba, University of Minnesota - MGIS Program

##### March 20, 2023

*Micromobility has been a recent interest of transportation planners and urban residents alike. One of the most popular implementations of micromobility in Minneapolis has been through motorized scooters from companies like Lime (Uber), Bird, or Lyft. There are comprehensive datasets available containing trip information for all scooter trips within the City, but using the data for understanding travel patterns and user behaviors has largely remained untouched. The data could have immense value to planners, residents, and other stakeholders, to inform future decision-making and more adequately prepare for how the urban landscape will change to accommodate greater volumes of scooters and other forms of micromobility. My solution is to use the powerful techniques of spatial data science to uncover these patterns and estimate when and where users are traveling **to** and **from**.*

In this notebook, we will be demonstrating a few of the many methods that can be used in ArcGIS Pro to perform spatiotemporal analytics. To see the ETL and data wrangling process, cheack out the first notebook, called *etl.ipynb*.

In [1]:
# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Libraries
import arcpy
import os

In [3]:
# Set Paths
origin_stc = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\origins_stc.nc"
destination_stc = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\destinations_stc.nc"

origin_daily_stc = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\originsDaily_stc.nc"
destination_daily_stc = r"C:\gitFiles\MicromobilityAnalytics\data\outputs\destinationsDaily_stc.nc"

stcs = [origin_stc, destination_stc]
daily_stcs = [origin_daily_stc, destination_daily_stc]

working_gdb = r"C:\gitFiles\MicromobilityAnalytics\MicromobilityAPRX\MicromobilityAPRX.gdb"

# Set as Working Envirnoment
arcpy.env.workspace = working_gdb

## Time Series Clustering

The first technique that will be explored is Time Series Clustering (TSC). For more information on how TSC works, check out the [documentation](https://pro.arcgis.com/en/pro-app/latest/tool-reference/space-time-pattern-mining/learnmoretimeseriesclustering.htm).

In this example, we are running TSC for all three characteristics of interest that are available: Value, Profile (Correlation), and Profile (Fourier). This is done automatically in the function below, to make the code less repetitive.

In [4]:
def automateTimeSeriesClustering(stc_list: list) -> None:
    """Automates the process of running TSC with different methods on a list of STCs.

    Args:
        stc_list (list): Input list of STC paths.
    """
    # Methods List
    methods = ["VALUE", "PROFILE", "PROFILE_FOURIER"]
    
    # Loop through Input STCs
    for stc in stc_list:
        # Get STC Name
        _, name = os.path.split(stc)
        od_name = name.split("_")[0]
        
        # Loop through Methods and Run Clustering
        for method in methods:
            out = arcpy.stpm.TimeSeriesClustering(
                stc,
                "COUNT",
                f"{od_name}_{method}_tsc",
                method,
                output_table_for_charts=f"{od_name}_{method}_tscTable",
                enable_time_series_popups="CREATE_POPUP"
            )

In [5]:
# Run Time Series Clustering on Half-Hour STCs
automateTimeSeriesClustering(stcs)

In [6]:
# Run Time Series Clustering on Daily STCs
automateTimeSeriesClustering(daily_stcs)

## Local Outlier Analysis

The second technique that will be used is Local Outlier Analysis (LOA). For more information on how LOA works, check out the [documentation](https://pro.arcgis.com/en/pro-app/latest/tool-reference/space-time-pattern-mining/learnmorelocaloutlier.htm).

In this example, only the daily STCs were used, and neighbors were calculated by using the K Nearest Neighbors algorithm, with 6 neighbors. Becuase the input dataset is hexagonal, this means that the 6 nearest neighbors should be the other features that border a given feature.

In [7]:
def automateLocalOutlierAnalysis(stc_list: list) -> None:
    """Automates the process of running LOA on a list of STCs.

    Args:
        stc_list (list): Input list of STC paths.
    """
    # Loop through STCs
    for stc in stc_list:
        # Get STC Name
        _, name = os.path.split(stc)
        od_name = name.split("_")[0]
        
        # Run LOA
        arcpy.stpm.LocalOutlierAnalysis(
            stc,
            "COUNT",
            os.path.join(working_gdb, f"{od_name}_LOA"),
            neighborhood_time_step = 1,
            number_of_permutations = 99,
            conceptualization_of_spatial_relationships = "K_NEAREST_NEIGHBORS",
            number_of_neighbors = 6,
            define_global_window = "ENTIRE_CUBE"
        )

In [8]:
# Run Local Outlier Analysis on Daily STCs
automateLocalOutlierAnalysis(daily_stcs)

## Emerging Hot Spot Analysis

The final technique that will be used is Emerging Hot Spot Analysis (EHSA). For more information on how EHSA works, check out the [documentation](https://pro.arcgis.com/en/pro-app/latest/tool-reference/space-time-pattern-mining/learnmoreemerging.htm).

Like the LOA example, neighbors weere determined by using the K Nearest Neighbors algorithm to find the 6 nearest neighbors.

In [9]:
def automateEmergingHotSpotAnalysis(stc_list: list) -> None:
    """Automates the process of running EHSA on a list of STCs.

    Args:
        stc_list (list): Input list of STC paths.
    """
    # Loop through STCs
    for stc in stc_list:
        # Get STC Name
        _, name = os.path.split(stc)
        od_name = name.split("_")[0]
        
        # Run EHSA
        arcpy.stpm.EmergingHotSpotAnalysis(
            stc,
            "COUNT",
            os.path.join(working_gdb, f"{od_name}_EHSA"),
            neighborhood_time_step = 1,
            conceptualization_of_spatial_relationships = "K_NEAREST_NEIGHBORS",
            number_of_neighbors = 6,
            define_global_window = "ENTIRE_CUBE"
        )

In [10]:
# Run Emerging Hot Spot Analysis on Half-Hour STCs
automateEmergingHotSpotAnalysis(stcs)

In [11]:
# Run Emerging Hot Spot Analysis on Daily STCs
automateEmergingHotSpotAnalysis(daily_stcs)

## The End!

This notebook provides a brief overview of only a few of the many techniques and methods that could be employed for spatiotemporal analysis or prediction tasks.

For more information on spatiotemporal tools that could be useful, check out the list below of several potential options.
- [Space Time Pattern Analysis](https://pro.arcgis.com/en/pro-app/latest/tool-reference/space-time-pattern-mining/an-overview-of-the-space-time-pattern-analysis-toolset.htm)
- [Time Series Forecasting](https://pro.arcgis.com/en/pro-app/latest/tool-reference/space-time-pattern-mining/an-overview-of-the-time-series-forecasting-toolset.htm)
- [Time Series AI](https://pro.arcgis.com/en/pro-app/latest/tool-reference/geoai/an-overview-of-the-time-series-ai-toolset.htm)
- [Temporal Joins](https://pro.arcgis.com/en/pro-app/latest/tool-reference/geoanalytics-desktop/join-features.htm)
- [Movement Analysis](https://pro.arcgis.com/en/pro-app/latest/tool-reference/intelligence/an-overview-of-the-movement-analysis-toolset.htm)
- [Multidimensional Data](https://pro.arcgis.com/en/pro-app/latest/tool-reference/multidimension/an-overview-of-the-multidimension-toolbox.htm)
- [Multidimensional Analysis](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/an-overview-of-the-multidimensional-analysis-tools.htm)

For more information on previous project I worked on, which used spatiotemporal data mining to examine the same data, check out this GitHub [repository](https://github.com/lukezaruba/MicromobilitySTDM).